In [11]:
import os

openai_api_key = os.getenv('OPENAI_API_KEY')

In [12]:
# Download poppler from here - https://github.com/oschwartz10612/poppler-windows/releases/


from pdf2image import convert_from_path


def pdf_to_img_converter(pdf_path: str, output_img_folder_path: str) -> None:

    os.makedirs(output_img_folder_path, exist_ok=True)

    images = convert_from_path(pdf_path, poppler_path=r"C:/Program Files (x86)/poppler-23.07.0/Library/bin")

    for i, image in enumerate(images):
        image.save(os.path.join(output_img_folder_path, f"image_{str(i)}.jpg"))

In [13]:
pdf_path = 'D:/text-summarization-using-langchain/chandrayan-3.pdf'
output_img_folder_path = 'D:/text-summarization-using-langchain/pdf_images/'

pdf_to_img_converter(pdf_path=pdf_path, output_img_folder_path=output_img_folder_path)

In [14]:
import pytesseract
import cv2


pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'


def img_to_txt_converter(output_txt_folder_path: str, img_pdf_folder_path: str) -> None:

    os.makedirs(output_txt_folder_path, exist_ok=True)

    for i, resume in enumerate(os.listdir(img_pdf_folder_path)):
        img = cv2.imread(img_pdf_folder_path + "/" + resume)
        img = cv2.resize(img, (1000, 1100))
        text = pytesseract.image_to_string(img)

        fname = os.path.join(output_txt_folder_path, f"txt_{str(i)}.txt")
        with open(fname, "w", encoding='utf-8') as file:
            file.write(text)

    

In [15]:
img_pdf_folder_path = 'D:/text-summarization-using-langchain/pdf_images/'
output_txt_folder_path = 'D:/text-summarization-using-langchain/pdf_text/'

img_to_txt_converter(output_txt_folder_path=output_txt_folder_path, img_pdf_folder_path=img_pdf_folder_path)